# 🐍 Python 语法进阶 - 综合练习

结合你学习的 **列表推导式**、**双端队列 (Deque)**、**栈 (Stack/RPN)** 和 **埃氏筛 (Primes)**，这里有 4 道不同难度的练习题来巩固你的掌握程度。

## 🟢 第一关：数据清洗大师 (列表推导式)

**背景**：你从后台 API 拿到了一组用户数据，但数据很脏。
**数据**：
```python
raw_data = [
    {"name": "  alice ", "age": 25, "score": 80},
    {"name": "Bob", "age": -5, "score": 90},  # 年龄无效
    {"name": "cindy", "age": 30, "score": 55},
    {"name": "David", "age": 150, "score": 100}, # 年龄无效
    {"name": "  eve", "age": 22, "score": 95}
]
```
**任务**：请用**一行列表推导式**完成以下操作：
1.  去掉年龄不合法的数据（合法范围：0 < age < 120）。
2.  把名字两边的空格去掉，并转成首字母大写 (Title Case)。
3.  提取出的结果只需要包含名字 (Name) 组成的列表。

**预期结果**：`['Alice', 'Cindy', 'Eve']`

In [2]:
raw_data = [
    {"name": "  alice ", "age": 25, "score": 80},
    {"name": "Bob", "age": -5, "score": 90},
    {"name": "cindy", "age": 30, "score": 55},
    {"name": "David", "age": 150, "score": 100},
    {"name": "  eve", "age": 22, "score": 95}
]

res = [i["name"].strip().title() for i in raw_data if i["age"]> 0 and i["age"] < 120]
print(res)

['Alice', 'Cindy', 'Eve']


## 🟡 第二关：有效括号 (栈 Stack)

**背景**：在编辑器中，括号必须成对出现且顺序正确。比如 `{[()]}` 是合法的，而 `([)]` 是非法的。
**任务**：写一个函数 `isValid(s: str) -> bool`，判断输入的字符串括号是否合法。
**要求**：
*   支持 `()`, `[]`, `{}` 三种括号。
*   使用 `list` 作为栈来解决。

**提示**：遇到左括号入栈，遇到右括号看栈顶是不是匹配的左括号。

In [7]:
def isValid(tokens):
    dic = {
        '(':')',
        '[':']',
        '{':'}',
    }
    stack = []
    for char in tokens:
        if char in dic:
            stack.append(char)
        else:
            if not stack:
                return False
            else:
                a = stack.pop()
                if dic[a] != char:
                    return False 
    return not stack

# 测试用例
print(isValid("()[]{}")) # True
print(isValid("(]"))     # False
print(isValid("([)]"))   # False
print(isValid("{[]}"))   # True

# --- 新增的边界测试 ---
print(isValid("]"))      # False (右括号多)
print(isValid("(()"))    # False (左括号多)
print(isValid("([{}])")) # True (复杂嵌套)
print(isValid(""))       # True (空字符串)

True
False
False
True
False
False
True
True


In [16]:
def isValid(tokens):
    dic = {
        ')':'(',
        '}':'{',
        ']':'[',
    }
    stack = []
    for char in tokens:
        if char in dic:
            if not stack:
                return False
            else:
                a = stack.pop()
                if a != dic[char]:
                    return False
        else:
            stack.append(char)
    return not stack

# 测试用例
print(isValid("()[]{}")) # True
print(isValid("(]"))     # False
print(isValid("([)]"))   # False
print(isValid("{[]}"))   # True

# --- 新增的边界测试 ---
print(isValid("]"))      # False (右括号多)
print(isValid("(()"))    # False (左括号多)
print(isValid("([{}])")) # True (复杂嵌套)
print(isValid(""))       # True (空字符串)

True
False
False
True
False
False
True
True


### 📝 学习复盘：栈 (Stack) 的避坑指南

通过这道括号匹配题，总结出我在处理数据结构题目时容易忽视的 3 个关键点：

1.  **先判空，再操作 (Check Before Action)**
    *   **错误习惯**：习惯性直接写 `stack.pop()`，默认栈里肯定有东西。
    *   **后果**：遇到非法输入（如 `"]"`）时，程序会因 `IndexError` 崩溃。
    *   **修正**：在 `pop` 或 `top` 之前，永远要问一句 `if not stack:`。

2.  **不要急着下结论 (Avoid Premature Return)**
    *   **错误习惯**：在循环里写 `if ...: return True`。只要匹配成功一对，就误以为全都通过了。
    *   **后果**：会漏掉后面的字符（如 `()[]` 只检查了 `()` 就结束了）。
    *   **修正**：**验证成功保持沉默**（继续循环），**验证失败立刻报错**（`return False`）。只有当所有关卡都跑完了，才能在函数最后放心地返回 `True`。

3.  **收尾检查不能忘 (Clean Up Check)**
    *   **错误习惯**：循环结束直接 `return True`，忘记检查容器里是否还有残留。
    *   **后果**：无法识别“左括号多余”的情况（如 `(()`）。
    *   **修正**：循环结束后，一定要确认容器（栈）是空的（`return not stack`）。

---
> **💡 心法口诀**：
> *   **操作前看一眼**（栈空了吗？）
> *   **坏消息早报**（不对直接 False）
> *   **好消息晚报**（跑完再 True）
> *   **出门查口袋**（栈清空了吗？）

## 🟠 第三关：最近的请求次数 (双端队列 Deque)

**背景**：我们需要计算最近 3000 毫秒内的请求数量。
**任务**：设计一个 `RecentCounter` 类：
*   `ping(t)`: 输入一个时间 `t` (毫秒)，返回在 `[t-3000, t]` 范围内的请求个数。
*   保证每次调用 `ping` 的 `t` 都是严格递增的。
*   **必须使用 `collections.deque`**，因为如果用列表，删除头部元素太慢了。

**示例**：
*   `ping(1)` -> 队列 `[1]`, 返回 1
*   `ping(100)` -> 队列 `[1, 100]`, 返回 2
*   `ping(3001)` -> 队列 `[1, 100, 3001]`, 返回 3
*   `ping(3002)` -> 队列 `[1, 100, 3001, 3002]`, 队头 `1` 超出范围 (3002-3000=2)，弹出。队列 `[100, 3001, 3002]`, 返回 3。

In [24]:
from collections import deque

class RecentCounter:
    def __init__(self):
        self.q = deque()

    def ping(self, t: int) -> int:
        self.q.append(t)
        while self.q[0] < t - 3000:
            self.q.popleft() 
        return len(self.q)

# 测试
counter = RecentCounter()
print(counter.ping(1))    # 1
print(counter.ping(100))  # 2
print(counter.ping(3001)) # 3
print(counter.ping(3002)) # 3

1
2
3
3


## 🔴 第四关：寻找 "孪生质数" (综合应用)

**背景**：结合**埃氏筛**和**列表推导式**。
**定义**：孪生质数是指相差为 2 的两个质数对，比如 (3, 5), (5, 7), (11, 13)。
**任务**：
1.  使用埃氏筛 (Sieve of Eratosthenes) 生成 100 以内的所有质数列表。
2.  使用列表推导式，从这个质数列表中找出所有的孪生质数对。
3.  输出格式：`[(3, 5), (5, 7), ...]`

In [33]:

n = 100
is_prime = [True] * (n + 1)
is_prime[0] = is_prime[1] = False
for i in range(2,int(n + 1)):
    if is_prime[i] == True:
        for j in range(i * i,n + 1,i):
            is_prime[j] = False

prime = [i for i in range(2,int(n + 1)) if is_prime[i] == True]
print(prime)
twin_primes = [(prime[i],prime[i + 1]) for i in range(0,int(len(prime) - 1)) if prime[i] + 2 == prime[i + 1]]
print(twin_primes)

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97]
[(3, 5), (5, 7), (11, 13), (17, 19), (29, 31), (41, 43), (59, 61), (71, 73)]


### 📝 学习复盘：埃氏筛 + 列表推导式的避坑指南

---

#### 🔴 埃氏筛部分

1.  **混淆 `n**2` 和 `n**0.5`**
    *   **错误**：`range(2, int(n**2 + 1))` —— n² = 10000，范围太大了
    *   **正确**：只需要遍历到 √n，即 `n**0.5`，或者简单点直接到 `n+1`

2.  **检查对象搞错**
    *   **错误**：`if i == True` —— 这是在问"i 是不是等于 1"
    *   **正确**：`if is_prime[i]` —— 应该检查"第 i 个数是不是质数"

3.  **混淆"标记"和"收集"**
    *   **错误**：在埃氏筛循环里写 `is_prime.append(i)`（往布尔数组塞整数）
    *   **正确**：埃氏筛只负责**标记**（填充布尔数组），收集质数应该**单独一步**

4.  **变量名拼写错误**
    *   **错误**：定义 `primt = []`，使用 `prime.append()`
    *   **教训**：变量名要统一，写完检查一遍拼写

---

#### 🟡 列表推导式部分

1.  **语法格式错误**
    *   **错误**：`[表达式, for ..., if 条件]`（用逗号分隔）
    *   **正确**：`[表达式 for 变量 in 范围 if 条件]`（用空格分隔）

2.  **收集的内容搞错**
    *   **错误**：`[is_prime[i] for ...]` —— 收集的是布尔值 True/False
    *   **正确**：`[i for ...]` —— 应该收集下标 i（即质数本身）

---

#### 🟠 索引边界部分

1.  **起点错误**
    *   **错误**：`range(1, ...)` —— Python 索引从 0 开始
    *   **正确**：`range(0, ...)` 或直接 `range(...)`

2.  **终点越界**
    *   **错误**：`range(..., len(prime) + 1)` —— 访问 `prime[len(prime)]` 会越界
    *   **正确**：因为要同时访问 `prime[i]` 和 `prime[i+1]`，所以 `i` 最大只能到 `len(prime) - 2`
    *   **公式**：`range(len(prime) - 1)`

---

> **💡 心法口诀**：
> *   **平方根不是平方**（√n vs n²）
> *   **检查数组值，不是检查下标**（`is_prime[i]` vs `i`）
> *   **标记和收集分两步**
> *   **往后看一个，提前停一步**（访问 `[i+1]` 就要 `len - 1`）